In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import cPickle
import os
import matplotlib
os.chdir('/Users/zbutler/research/fire_prediction')
from data import data
import prediction.cluster_regression as cr
from prediction.cluster_regression import ClusterRegression
from metrics.evaluation import evaluate_glm, cross_validation_evaluation
%matplotlib inline

In [ ]:
clust_feat_df = data.load_clust_feat_df(clust_thresh=5)

In [ ]:
# Get metrics for zero padding
import metrics.evaluation as ev
reload(ev)
reload(cr)
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
mets = ("MSE", "RobustMSE", "MeanAbsErr")

cdf_with_stuff = cr.add_autoreg_and_n_det(clust_feat_df.copy(), autoreg_cols=1, t_k_max=6, zero_padding=True)
print list(cdf_with_stuff.iloc[0:10]["t_k_4"])
for t_k in range(5,-1,-1):
    y_t_k_base = cdf_with_stuff["t_k_%d" % t_k]
    y_hat_t_k_base = cdf_with_stuff["autoreg_1"]
    my_mets = []
    for met in mets:
        my_mets.append(evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    mets_baseline[t_k] = my_mets
    
    mets_mem[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=[], zero_padding=True, return_arrs=False)
    mets_temphum[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=True, return_arrs=False)
    mets_weather[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=True, return_arrs=False)
    print "done with t_k %d" % t_k
print mets_mem

In [ ]:
n_t_k = 6
t_k_arr = range(n_t_k)
for i,met in enumerate(mets):
    base_arr = np.array(map(lambda x: mets_baseline[x][i], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr, base_arr, "k--", label="Baseline")
    plt.plot(t_k_arr, mem_arr, "g--", label="Memory")
    plt.plot(t_k_arr, temphum_arr, "r--", label="Temp/hum")
    plt.plot(t_k_arr, weather_arr, "b--", label="Weather")
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.savefig("pics/t_k_zero_padding_%s.png" % met)
    plt.show()
    plt.close()

In [ ]:
# Get metrics for nan padding
import metrics.evaluation as ev
reload(ev)
reload(cr)
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
mets = ("MSE", "RobustMSE", "MeanAbsErr")
n_ex = []

print 'yolo'
#cdf_with_stuff = cr.add_autoreg_and_n_det(clust_feat_df.copy(), autoreg_cols=1, t_k_max=6, zero_padding=False)
print list(cdf_with_stuff.iloc[0:10]["t_k_4"])
for t_k in range(5,-1,-1):
    y_t_k_base = cdf_with_stuff["t_k_%d" % t_k]
    y_hat_t_k_base = cdf_with_stuff["autoreg_1"]
    my_mets = []
    for met in mets:
        my_mets.append(ev.evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    mets_baseline[t_k] = my_mets
    
    mets_mem[t_k],y,y_hat = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=[], zero_padding=False, return_arrs=True)
    mets_temphum[t_k],y,y_hat = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=False, return_arrs=True)
    mets_weather[t_k],y,y_hat = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=True)
    
    n_ex.append(np.sum(~np.isnan(y_t_k_base)))
    print "done with t_k %d" % t_k

In [ ]:
n_t_k = 6
t_k_arr = range(n_t_k)
print mets_baseline
plt.plot(t_k_arr, n_ex[::-1], 'r--')
plt.xlabel("t_k")
plt.ylabel("Number of training examples")
plt.show()
plt.close()

for i,met in enumerate(mets):
    base_arr = np.array(map(lambda x: mets_baseline[x][i], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr, base_arr, "k--", label="Baseline")
    plt.plot(t_k_arr, mem_arr, "g--", label="Memory")
    plt.plot(t_k_arr, temphum_arr, "r--", label="Temp/hum")
    plt.plot(t_k_arr, weather_arr, "b--", label="Weather")
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.savefig("pics/t_k_zero_padding_%s.png" % met)
    plt.show()
    plt.close()

In [ ]:
# Now do it again but don't be a moron
# Get metrics for zero padding
import metrics.evaluation as ev
reload(ev)
reload(cr)
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
mets = ("MSE", "RobustMSE", "MeanAbsErr")

cdf_with_stuff = cr.add_autoreg_and_n_det(clust_feat_df.copy(), autoreg_cols=10, t_k_max=0, zero_padding=False)
for t_k in range(5,-1,-1):
    y_t_k_base = cdf_with_stuff['n_det']
    y_hat_t_k_base = np.exp(cdf_with_stuff["autoreg_%d" % (1 + t_k)]) - 1
    my_mets = []
    for met in mets:
        my_mets.append(evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    mets_baseline[t_k] = my_mets
    
    mets_mem[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=[], zero_padding=True, return_arrs=False)
    mets_temphum[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=True, return_arrs=False)
    mets_weather[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=True, return_arrs=False)
    print "done with t_k %d" % t_k
print mets_mem

In [ ]:
n_t_k = 6
t_k_arr = range(n_t_k)
print mets_baseline

for i,met in enumerate(mets):
    base_arr = np.array(map(lambda x: mets_baseline[x][i], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr, base_arr, "k--", label="Baseline")
    plt.plot(t_k_arr, mem_arr, "g--", label="Memory")
    plt.plot(t_k_arr, temphum_arr, "r--", label="Temp/hum")
    plt.plot(t_k_arr, weather_arr, "b--", label="Weather")
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.savefig("pics/t_k_zero_padding_%s.png" % met)
    plt.show()
    plt.close()

In [ ]:
def kill_nanners(df, weather_vars):
    for cov in weather_vars:
        nanners = df[np.isnan(df[cov])]
        for name in nanners.index:
            clust = nanners.loc[name].cluster
            dayofyear = nanners.loc[name].dayofyear
            next_offset_to_try = -1
            while 1:
                # If offset is getting too far away, just replace with mean across all time
                if abs(next_offset_to_try) > 5:
                    df.set_value(name, cov, np.mean(df[cov]))
                    break
                pot_val = df[(df.dayofyear==(dayofyear+next_offset_to_try)) & (df.cluster==clust)][cov]
                if not len(pot_val) or np.isnan(float(pot_val)):
                    if next_offset_to_try < 0:
                        next_offset_to_try = - next_offset_to_try
                    else:
                        next_offset_to_try = - (next_offset_to_try + 1)
                else:
                    df.set_value(name, cov, float(pot_val))
                    break
    return df

In [ ]:
# Now do it again but don't be a moron
# Get metrics for zero padding
import metrics.evaluation as ev
reload(ev)
reload(cr)
mets_baseline = dict()
mets_mem = dict()
mets_temphum = dict()
mets_weather = dict()
mets_only_weather = dict()
mets = ("MSE", "RobustMSE", "MeanAbsErr")
print "hi"
#cdf_with_stuff = cr.add_autoreg_and_n_det(clust_feat_df.copy(), autoreg_cols=10, t_k_max=0, zero_padding=False)
#cdf_with_stuff = kill_nanners(cdf_with_stuff.copy(), ["temp", "humidity", "wind", "rain_del_2"])
max_t_k = 4

for t_k in range(max_t_k,-1,-1):
    legit_series = pd.Series(index=cdf_with_stuff.index)
    for clust in cdf_with_stuff.cluster.unique():
        clust_df = cdf_with_stuff[cdf_with_stuff.cluster==clust]
        legit_day = np.min(clust_df.dayofyear) + t_k
        legit_series[clust_df[clust_df.dayofyear >= legit_day].index] = 1
    y_t_k_base = cdf_with_stuff[legit_series == 1]['n_det']
    y_hat_t_k_base = np.exp(cdf_with_stuff[legit_series == 1]["autoreg_%d" % (1 + t_k)]) - 1
    my_mets = []
    for met in mets:
        my_mets.append(evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
    mets_baseline[t_k] = my_mets
    
    mets_mem[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=[], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series)
    mets_temphum[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series)
    mets_weather[t_k] = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series)
    mets_only_weather[t_k]= ev.cross_validation_evaluation(cdf_with_stuff, autoreg=0, t_k=t_k, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=False, max_t_k=t_k, legit_series=legit_series)
    print "done with t_k %d" % t_k
print mets_mem

In [ ]:
n_t_k = 5
t_k_arr = np.arange(n_t_k)
mets = ["MeanAbsErr"]
i = 2
for met in mets:
    base_arr = np.array(map(lambda x: mets_baseline[x][i], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    only_weather_arr = np.array(map(lambda x: mets_only_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr+1, base_arr, "kv--", label="Baseline", linewidth=2)
    plt.plot(t_k_arr+1, mem_arr, "gs--", label="Autoregression", linewidth=2)
    plt.plot(t_k_arr+1, temphum_arr, "r^--", label="Temp/hum", linewidth=2)
    plt.plot(t_k_arr+1, weather_arr, "bo--", label="All weather", linewidth=2)
    #plt.plot(t_k_arr+1, only_weather_arr, "m--", label="Only weather")
#plt.axhline(y=mets_only_weather[i], color="m", label="Weather only")
    
    matplotlib.rcParams.update({'font.size': 14})
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Day of forecast (k)")
    plt.xticks(t_k_arr+1)
    plt.ylabel("Mean absolute error (2007-2016)")
    plt.savefig("pics/mae.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
for met in mets:
    base_arr = np.array(map(lambda x: mets_baseline[x][i], t_k_arr))
    mem_arr = np.array(map(lambda x: mets_mem[x][i], t_k_arr))
    temphum_arr = np.array(map(lambda x: mets_temphum[x][i], t_k_arr))
    weather_arr = np.array(map(lambda x: mets_weather[x][i], t_k_arr))
    only_weather_arr = np.array(map(lambda x: mets_only_weather[x][i], t_k_arr))
    
    plt.plot(t_k_arr+1, base_arr / base_arr, "k--", label="Baseline")
    plt.plot(t_k_arr+1, mem_arr / base_arr, "g--", label="Memory")
    plt.plot(t_k_arr+1, temphum_arr / base_arr, "r--", label="Temp/hum")
    plt.plot(t_k_arr+1, weather_arr / base_arr, "b--", label="Weather")
    plt.plot(t_k_arr+1, only_weather_arr / base_arr, "m--", label="Weather only")
    
    lgd = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Days into the future we are predicting")
    plt.xticks(t_k_arr+1)
    plt.ylabel("Mean absolute error (relative to baseline)")
    plt.savefig("pics/mae_rel_only_weather.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
# Debugging, what's up with MeanAbsErr?
# First, let's train the zero-padded models on all but 2013 and see what we get for
y_t_k_base = cdf_with_stuff['n_det']
y_hat_t_k_base = np.exp(cdf_with_stuff["autoreg_%d" % 1]) - 1
my_mets = []
for met in mets:
    my_mets.append(evaluate_glm(y_t_k_base, y_hat_t_k_base, metric=met))
mets_mem_z,y_mem,y_hat_mem = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=0, weather_vars=[], zero_padding=False, return_arrs=True)
mets_temphum_z,y_temp,y_hat_temp = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=0, weather_vars=["temp","humidity"], zero_padding=False, return_arrs=True)
mets_weather_z,y_weather,y_hat_weather = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=0, weather_vars=["temp","humidity","wind","rain_del_2"], zero_padding=False, return_arrs=True)
print "baseline_mets: " + str(my_mets)
plt.scatter(y_t_k_base,y_hat_t_k_base)
plt.xlabel("y")
plt.ylabel("y_hat")
plt.title("Baseline")
plt.show()
plt.close()

print "memory_mets: " + str(mets_mem_z)
plt.scatter(y_mem,y_hat_mem)
plt.xlabel("y")
plt.ylabel("y_hat")
plt.title("Memory")
plt.show()
plt.close()

print "temp/hum_mets: " + str(mets_temphum_z)
plt.scatter(y_temp,y_hat_temp)
plt.xlabel("y")
plt.ylabel("y_hat")
plt.title("Temp/hum")
plt.show()
plt.close()

print "weather_mets: " + str(mets_weather_z)
plt.scatter(y_weather,y_hat_weather)
plt.xlabel("y")
plt.ylabel("y_hat")
plt.title("Weather")
plt.show()
plt.close()

In [ ]:
# OK, how is memory doing so shittily compared to the baseline wrt mean abs error?
plt.scatter(np.abs(y_hat_mem - y_mem), np.abs(y_hat_t_k_base - y_mem))
plt.title("mean abs err scatter plot")
plt.show()

plt.scatter((y_hat_mem - y_mem)**2, (y_hat_t_k_base - y_mem)**2)
plt.title("mse scatter plot")
plt.show()

In [ ]:
reload(ev)
mets_mem_z,y_mem,y_hat_mem = ev.cross_validation_evaluation(cdf_with_stuff, autoreg=1, t_k=0, weather_vars=[], zero_padding=False, return_arrs=True, print_covars=True)

In [ ]:
np.mean(clust_feat_df.n_det)

In [ ]:
np.sum(legit_series == 1)

In [ ]:
#from prediction.cluster_regression import add_autoreg_and_n_det
#cdf = add_autoreg_and_n_det(clust_feat_df, 1,0)
plt.scatter(np.exp(cdf.autoreg_1)-1, cdf.n_det)
plt.axhline(y=0.0, color='k')
plt.axvline(x=0.0, color='k')
plt.xlabel("Counts at time t")
plt.ylabel("Counts at time t+1")
plt.show()

print "zero days: %d of %d" % (len(cdf[cdf.n_det == 0]), len(cdf))

cut_pts = [0, 1, 10, 100, 1000]
for i in xrange(len(cut_pts)-1):
    my_pts = cdf[(np.exp(cdf.autoreg_1)-1 >= cut_pts[i]) & (np.exp(cdf.autoreg_1)-1 < cut_pts[i+1])]
    print "For clusters and days with [%d,%d) detections (%d such days), we have on average %f detections tomorrow" % (cut_pts[i], cut_pts[i+1], len(my_pts), np.mean(my_pts.n_det))